# OrbitalShield Exploratory Data Analysis

In [38]:
# !pip install spacetrack

In [39]:
# !pip install sgp4

In [40]:
from collections import defaultdict
from datetime import(
    datetime,
    timedelta,
    timezone,
)
import getpass
import json
from typing import(
    Dict,
    List,
    Tuple,
)

import numpy as np
from numpy.linalg import norm
import pandas as pd
from scipy.spatial import Voronoi
from sgp4.api import(
    jday,
    Satrec,
    SatrecArray,
)
from spacetrack import SpaceTrackClient

## Extract latest data

In [41]:
# Define Space-Track credential
ST_USER = input("Username:")
ST_PWD = getpass.getpass("Password:")

In [42]:
# Fetch latest satellite records
with SpaceTrackClient(ST_USER, ST_PWD) as st:
    resp = st.gp(
        epoch=">now-1",
        format="json",
    )

In [43]:
# Convert JSON records to dataframe
df = pd.DataFrame(json.loads(resp))
df.head(2)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,...,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,3.0,GENERATED VIA SPACE-TRACK.ORG API,2025-05-13T04:54:31,18 SPCS,STARLINK-5635,2023-013AN,EARTH,TEME,UTC,SGP4,...,LARGE,US,2023-01-26,AFETR,None,4726983,287425155,0 STARLINK-5635,1 55367U 23013AN 25132.65572213 .00000533 0...,2 55367 43.0045 123.2521 0001228 261.4400 98...
1,3.0,GENERATED VIA SPACE-TRACK.ORG API,2025-05-13T04:54:31,18 SPCS,COSMOS 1275 DEB,1981-053LG,EARTH,TEME,UTC,SGP4,...,SMALL,CIS,1981-06-04,PKMTR,None,4726983,287407271,0 COSMOS 1275 DEB,1 17637U 81053LG 25132.65600970 .00001175 0...,2 17637 83.0679 91.8678 0129388 267.7880 273...


## Explore columns

In [ ]:
# Print record
cols = df.columns
records = df.loc[df["NORAD_CAT_ID"]=="25544"].iloc[0,:].tolist()

for i, col in enumerate(cols):
    print(col, ":", records[i])

CCSDS_OMM_VERS : 3.0
COMMENT : GENERATED VIA SPACE-TRACK.ORG API
CREATION_DATE : 2025-05-13T15:06:32
ORIGINATOR : 18 SPCS
OBJECT_NAME : ISS (ZARYA)
OBJECT_ID : 1998-067A
CENTER_NAME : EARTH
REF_FRAME : TEME
TIME_SYSTEM : UTC
MEAN_ELEMENT_THEORY : SGP4
EPOCH : 2025-05-13T10:40:15.402144
MEAN_MOTION : 15.49506546
ECCENTRICITY : 0.00023070
INCLINATION : 51.6344
RA_OF_ASC_NODE : 119.1760
ARG_OF_PERICENTER : 106.2285
MEAN_ANOMALY : 253.8958
EPHEMERIS_TYPE : 0
CLASSIFICATION_TYPE : U
NORAD_CAT_ID : 25544
ELEMENT_SET_NO : 999
REV_AT_EPOCH : 50977
BSTAR : 0.00016281000000
MEAN_MOTION_DOT : 0.00008689
MEAN_MOTION_DDOT : 0.0000000000000
SEMIMAJOR_AXIS : 6796.306
PERIOD : 92.933
APOAPSIS : 419.738
PERIAPSIS : 416.603
OBJECT_TYPE : PAYLOAD
RCS_SIZE : LARGE
COUNTRY_CODE : ISS
LAUNCH_DATE : 1998-11-20
SITE : TTMTR
DECAY_DATE : None
FILE : 4727603
GP_ID : 287448349
TLE_LINE0 : 0 ISS (ZARYA)
TLE_LINE1 : 1 25544U 98067A   25133.44462271  .00008689  00000-0  16281-3 0  9996
TLE_LINE2 : 2 25544  51.6344 

### Metadata columns

In [ ]:
# Print satellite's static metadata
df.loc[df["NORAD_CAT_ID"]=="25544"][[
    "NORAD_CAT_ID",
    "OBJECT_ID",  # International Code
    "OBJECT_NAME",
    "OBJECT_TYPE",
    "CLASSIFICATION_TYPE",
    "RCS_SIZE",  # Radar cross section size
    "COUNTRY_CODE",
    "LAUNCH_DATE",
    "SITE",  # Launch site
]].iloc[0,:]

NORAD_CAT_ID                 25544
OBJECT_ID                1998-067A
OBJECT_NAME            ISS (ZARYA)
OBJECT_TYPE                PAYLOAD
CLASSIFICATION_TYPE              U
RCS_SIZE                     LARGE
COUNTRY_CODE                   ISS
LAUNCH_DATE             1998-11-20
SITE                         TTMTR
Name: 15955, dtype: object

### TLE (Two-Line Element) columns

In [ ]:
# Print TLE (Two-Line Element)
df[["TLE_LINE1", "TLE_LINE2"]].iloc[0,:]

TLE_LINE1    1 55367U 23013AN  25132.65572213  .00000533  0...
TLE_LINE2    2 55367  43.0045 123.2521 0001228 261.4400  98...
Name: 0, dtype: object

## Propagate orbits

In [47]:
# Define array of satellite objects based on TLE data
sat_arry = SatrecArray([
    Satrec.twoline2rv(t1, t2) for t1, t2 in df[["TLE_LINE1", "TLE_LINE2"]].to_numpy()
])

In [48]:
# Define future timestamps to propagate orbits
diff_min = 5
total_min = 60 * 24
curr_ts = datetime.now(timezone.utc)
future_ts_li = []
for i in range(diff_min, total_min + diff_min, diff_min):
    ts = curr_ts + timedelta(minutes=i)
    future_ts_li.append(ts)

# Convert timestamps to Julian dates
jd_arry = fr_arry = np.empty(0)
for ts in future_ts_li:
    jd, fr = jday(
        ts.year,
        ts.month,
        ts.day,
        ts.hour,
        ts.minute,
        ts.second + ts.microsecond / 1e6
    )
    jd_arry = np.append(jd_arry, jd)
    fr_arry = np.append(fr_arry, fr)

# Propagate orbits using future dates
e, r, v = sat_arry.sgp4(jd_arry, fr_arry)  # error, position[x, y, z], velocity

In [125]:
# Remove satellites with invalid results
err_idxes = set()
for i, errs in enumerate(e):
    for err in errs:
        if err != 0:
            err_idxes.add(i)
err_idxes = list(err_idxes)
err_idxes.sort(reverse=True)
for idx in err_idxes:
    e = np.vstack((e[:idx], e[idx+1:]))
    r = np.vstack((r[:idx], r[idx+1:]))
    v = np.vstack((v[:idx], v[idx+1:]))

In [126]:
# Define dict for satellite IDs and their propagated orbits
"""
sat_orbit_dict = {
    sat_id_1: [
        [x_t1, y_t1, z_t1],
        ...
        [x_tn, y_tn, z_tn],
    ],
    ...
    sat_id_n:...
}
"""
sat_orbit_dict = dict(zip(df["NORAD_CAT_ID"], r))

## CA (Conjunction Assess)

In [ ]:
# Loop through future timestamps to find conjunction candidates
# at TCA (Time of Closest Approach) with miss distance under threshold
dist_threshold = 5  # km
sat_ids = list(sat_orbit_dict.keys())
pair_ts_dist_dict = defaultdict(list)
for i, ts in enumerate(future_ts_li):
    # Collect satellite positions at iterating timestamp
    positions = []
    for sat_id, orbits in sat_orbit_dict.items():
        positions.append(orbits[i])

    # Map Voronoi diagram to identify neighbouring satellites
    vor = Voronoi(positions)
    for idx1, idx2 in vor.ridge_points:
        sat1 = sat_ids[idx1]
        sat2 = sat_ids[idx2]

        # Collect pairs with Euclidean distance < threshold
        dist = norm(positions[idx1] - positions[idx2])
        if dist < dist_threshold:
            pair = tuple(sorted((sat1, sat2)))
            pair_ts_dist_dict[pair].append((ts, dist))

# Determine miss distance at TCA for conjunction candidates
ca_summary = []
for pair, ts_dist_li in pair_ts_dist_dict.items():
    if not ts_dist_li:
        continue

    tca, dist = min(ts_dist_li, key=lambda x: x[1])
    ca_summary.append({
        "Satellite 1": pair[0],
        "Satellite 2": pair[1],
        "TCA": tca,
        "Miss Distance (km)": dist,
    })

In [178]:
ca_summary

[{'Satellite 1': '48466',
  'Satellite 2': '60513',
  'TCA': datetime.datetime(2025, 5, 13, 15, 49, 6, 794282, tzinfo=datetime.timezone.utc),
  'Miss Distance (km)': 1.0152862226866495},
 {'Satellite 1': '5203',
  'Satellite 2': '54126',
  'TCA': datetime.datetime(2025, 5, 13, 17, 39, 6, 794282, tzinfo=datetime.timezone.utc),
  'Miss Distance (km)': 1.3967836988965352},
 {'Satellite 1': '31097',
  'Satellite 2': '62611',
  'TCA': datetime.datetime(2025, 5, 13, 16, 19, 6, 794282, tzinfo=datetime.timezone.utc),
  'Miss Distance (km)': 2.131428731945432},
 {'Satellite 1': '44628',
  'Satellite 2': '58038',
  'TCA': datetime.datetime(2025, 5, 13, 16, 29, 6, 794282, tzinfo=datetime.timezone.utc),
  'Miss Distance (km)': 4.984407326134472},
 {'Satellite 1': '20889',
  'Satellite 2': '43082',
  'TCA': datetime.datetime(2025, 5, 13, 16, 39, 6, 794282, tzinfo=datetime.timezone.utc),
  'Miss Distance (km)': 4.57801835634536},
 {'Satellite 1': '51802',
  'Satellite 2': '60890',
  'TCA': datetime.

## COLA (Collision Avoidance)

In [183]:
# Filter ca summary down to payload satellites
payload_sat_ids = df.loc[
    df["OBJECT_TYPE"].str.upper().str.strip() == "PAYLOAD"
]["NORAD_CAT_ID"].unique().tolist()

ca_summary_payload = [
    pair for pair in ca_summary
    if (pair["Satellite 1"] in payload_sat_ids)
    | (pair["Satellite 2"] in payload_sat_ids)
]

In [184]:
# Extract histroical data of conjunction candidates
cola_sat_ids = list(set(
    [pair["Satellite 1"] for pair in ca_summary_payload]
    + [pair["Satellite 2"] for pair in ca_summary_payload]
))

with SpaceTrackClient(ST_USER, ST_PWD) as st:
    resp2 = st.gp_history(
        norad_cat_id=cola_sat_ids,
        epoch=">now-14",
        format="json",
    )

In [185]:
# Convert JSON records to dataframe
df2 = pd.DataFrame(json.loads(resp2))
df2.head(1)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,...,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2.0,GENERATED VIA SPACE-TRACK.ORG API,2025-05-02T01:16:34,18 SPCS,ECHO 1 DEB (METAL OBJ),1960-009C,EARTH,TEME,UTC,SGP4,...,MEDIUM,US,1960-08-12,AFETR,None,4716072,286583390,0 ECHO 1 DEB (METAL OBJ),1 51U 60009C 25121.78334842 -.00000042 0...,2 51 47.2118 242.3744 0107174 54.2919 306...


In [186]:
# Check if all satellites are included in the fetched data
set(cola_sat_ids).difference(set(df2["NORAD_CAT_ID"].unique()))

set()

## Etc.

In [129]:
# Check SGP4 errors
from sgp4.api import SGP4_ERRORS
SGP4_ERRORS

{1: 'mean eccentricity is outside the range 0.0 to 1.0',
 2: 'nm is less than zero',
 3: 'perturbed eccentricity is outside the range 0.0 to 1.0',
 4: 'semilatus rectum is less than zero',
 5: '(error 5 no longer in use; it meant the satellite was underground)',
 6: 'mrt is less than 1.0 which indicates the satellite has decayed'}